### 參考資料
* https://medium.com/@mengchiang000/redis-cache淺談-應用-7513a9f5f2dd
* http://tw511.com/18/146/4292.html
* https://www.runoob.com/w3cnote/python-redis-intro.html
* https://kknews.cc/zh-tw/code/ljx6xke.html

In [58]:
import redis # !pip install redis

In [59]:
# 讀取本地端起的 redis-server
# redis 取出的结果默认是字节，我们可以设定 decode_responses=True 改成字符串。
r = redis.Redis(host='127.0.0.1', port=6379, decode_responses=True)

In [6]:
r.set('mykey', 10) #新增
print(r.get('mykey')) #獲取

10


In [7]:
r.set('tmp', 1234, ex=3) # key是"tmp" value是1234。將鍵值存入緩存, ex 可以設定過期的時間(秒)

True

In [9]:
print(r.get('tmp'))  # tmp=3 這個值過3秒就會不見

None


* 計算雜湊值

In [10]:
#!/usr/bin/python3
import hashlib
m = hashlib.md5()
data = "G. T. Wang"

# 先將資料編碼，再更新 MD5 雜湊值
m.update(data.encode("utf-8"))

h = m.hexdigest()
print(h)

6871bb74e606dc7b4988b643e4870a8d


In [28]:
import random
account = str(random.randint(1,10))
m = hashlib.md5()
m.update(account.encode("utf-8"))
password = m.hexdigest()
print(account, password)

6 1679091c5a880faf6fb5e6087eb1b2dc


* pymysql 連進本地資料庫

In [60]:
import pymysql
import pandas as pd
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='', db= 'test', charset='utf8')
cursor = conn.cursor()
sql = f"""
SELECT *
FROM test.student
"""
df = pd.read_sql(sql, conn)
df

,course,mark,name
0,Maths,60,Roger
1,Maths,60,Jerry
2,Maths,70,Ariel
3,Maths,55,Tracey
4,Biology,60,Tracey
5,Biology,70,Jerry
6,Econ,90,Jerry
7,Econ,60,Ariel


In [33]:
sql = f"""
INSERT INTO test.student(course, mark, name) VALUES ("123", "456", "789")
"""
cursor.execute(sql)
conn.commit()

***

In [61]:
r.flushdb() # 實驗一開始先清除 redis cache
trun_cmd = f"""TRUNCATE TABLE test.tb_user"""
cursor.execute(trun_cmd) # 實驗一開始先清除 mysql database
conn.commit()

# 100 個使用者; 隨機進來登入 10000次
for i in range(1,10000):
    account = str(random.randint(1,100))
    m = hashlib.md5()
    m.update(account.encode("utf-8"))
    password = m.hexdigest()
    
    # 先去 redis 找
    pw = r.get(account)
    if not pw:
        # redis 找不到就先去 mysql 找
        sql = f"""
        SELECT password FROM test.tb_user WHERE account={account}
        """
        df = pd.read_sql(sql, conn)
        if len(df)==0:
            # 如果 mysql 找不到，就寫一筆進去 database
            cmd = f"""
            INSERT INTO test.tb_user(account, password) VALUES ("{account}", "{password}")
            """
            cursor.execute(cmd)
            conn.commit()
            # 再寫一筆進去 redis
            r.set(account, password, ex=10000)
        else:
            # mysql 有找到，一樣要寫到 redis
            r.set(account, password, ex=10000)
    
    
    print(account, password)

KeyboardInterrupt: 

In [57]:
r.flushdb() # 實驗一開始先清除 redis cache
trun_cmd = f"""TRUNCATE TABLE test.tb_user"""
cursor.execute(trun_cmd) # 實驗一開始先清除 mysql database
conn.commit()

# 100 個使用者; 隨機進來登入 10000次
for i in range(1,10000):
    account = str(random.randint(1,100))
    m = hashlib.md5()
    m.update(account.encode("utf-8"))
    password = m.hexdigest()
    
    # 每次都直接去 mysql 找
    sql = f"""
    SELECT password FROM test.tb_user WHERE account={account}
    """
    df = pd.read_sql(sql, conn)
    if len(df)==0:
        # 如果 mysql 找不到，就寫一筆進去 database
        cmd = f"""
        INSERT INTO test.tb_user(account, password) VALUES ("{account}", "{password}")
        """
        cursor.execute(cmd)
        conn.commit()
        # 再寫一筆進去 redis
    

    print(account, password)

30 34173cb38f07f89ddbebc2ac9128303f
70 7cbbc409ec990f19c78c75bd1e06f215
5 e4da3b7fbbce2345d7772b0674a318d5
71 e2c420d928d4bf8ce0ff2ec19b371514
6 1679091c5a880faf6fb5e6087eb1b2dc
84 68d30a9594728bc39aa24be94b319d21
70 7cbbc409ec990f19c78c75bd1e06f215
63 03afdbd66e7929b125f8597834fa83a4
92 92cc227532d17e56e07902b254dfad10
14 aab3238922bcc25a6f606eb525ffdc56
20 98f13708210194c475687be6106a3b84
69 14bfa6bb14875e45bba028a21ed38046
42 a1d0c6e83f027327d8461063f4ac58a6
91 54229abfcfa5649e7003b83dd4755294
15 9bf31c7ff062936a96d3c8bd1f8f2ff3
93 98dce83da57b0395e163467c9dae521b
16 c74d97b01eae257e44aa9d5bade97baf
98 ed3d2c21991e3bef5e069713af9fa6ca
37 a5bfc9e07964f8dddeb95fc584cd965d
16 c74d97b01eae257e44aa9d5bade97baf
66 3295c76acbf4caaed33c36b1b5fc2cb1
52 9a1158154dfa42caddbd0694a4e9bdc8
89 7647966b7343c29048673252e490f736
97 e2ef524fbf3d9fe611d5a8e90fefdc9c
10 d3d9446802a44259755d38e6d163e820
58 66f041e16a60928b05a7e228a89c3799
84 68d30a9594728bc39aa24be94b319d21
69 14bfa6bb14875e45bba028a21ed

24 1ff1de774005f8da13f42943881c655f
94 f4b9ec30ad9f68f89b29639786cb62ef
16 c74d97b01eae257e44aa9d5bade97baf
74 ad61ab143223efbc24c7d2583be69251
15 9bf31c7ff062936a96d3c8bd1f8f2ff3
36 19ca14e7ea6328a42e0eb13d585e4c22
54 a684eceee76fc522773286a895bc8436
46 d9d4f495e875a2e075a1a4a6e1b9770f
67 735b90b4568125ed6c3f678819b6e058
6 1679091c5a880faf6fb5e6087eb1b2dc
29 6ea9ab1baa0efb9e19094440c317e21b
32 6364d3f0f495b6ab9dcf8d3b5c6e0b01
100 f899139df5e1059396431415e770c6dd
17 70efdf2ec9b086079795c442636b55fb
89 7647966b7343c29048673252e490f736
50 c0c7c76d30bd3dcaefc96f40275bdc0a
22 b6d767d2f8ed5d21a44b0e5886680cb9
1 c4ca4238a0b923820dcc509a6f75849b
24 1ff1de774005f8da13f42943881c655f
46 d9d4f495e875a2e075a1a4a6e1b9770f
79 d1fe173d08e959397adf34b1d77e88d7
72 32bb90e8976aab5298d5da10fe66f21d
95 812b4ba287f5ee0bc9d43bbf5bbe87fb
84 68d30a9594728bc39aa24be94b319d21
13 c51ce410c124a10e0db5e4b97fc2af39
27 02e74f10e0327ad868d138f2b4fdd6f0
15 9bf31c7ff062936a96d3c8bd1f8f2ff3
62 44f683a84163b3523afe57c2e0

55 b53b3a3d6ab90ce0268229151c9bde11
53 d82c8d1619ad8176d665453cfb2e55f0
33 182be0c5cdcd5072bb1864cdee4d3d6e
8 c9f0f895fb98ab9159f51fd0297e236d
94 f4b9ec30ad9f68f89b29639786cb62ef
31 c16a5320fa475530d9583c34fd356ef5
1 c4ca4238a0b923820dcc509a6f75849b
71 e2c420d928d4bf8ce0ff2ec19b371514
88 2a38a4a9316c49e5a833517c45d31070
89 7647966b7343c29048673252e490f736
47 67c6a1e7ce56d3d6fa748ab6d9af3fd7
44 f7177163c833dff4b38fc8d2872f1ec6
69 14bfa6bb14875e45bba028a21ed38046
55 b53b3a3d6ab90ce0268229151c9bde11
27 02e74f10e0327ad868d138f2b4fdd6f0
77 28dd2c7955ce926456240b2ff0100bde
64 ea5d2f1c4608232e07d3aa3d998e5135
25 8e296a067a37563370ded05f5a3bf3ec
12 c20ad4d76fe97759aa27a0c99bff6710
51 2838023a778dfaecdc212708f721b788
10 d3d9446802a44259755d38e6d163e820
52 9a1158154dfa42caddbd0694a4e9bdc8
44 f7177163c833dff4b38fc8d2872f1ec6
80 f033ab37c30201f73f142449d037028d
11 6512bd43d9caa6e02c990b0a82652dca
14 aab3238922bcc25a6f606eb525ffdc56
28 33e75ff09dd601bbe69f351039152189
48 642e92efb79421734881b53e1e1

61 7f39f8317fbdb1988ef4c628eba02591
33 182be0c5cdcd5072bb1864cdee4d3d6e
94 f4b9ec30ad9f68f89b29639786cb62ef
44 f7177163c833dff4b38fc8d2872f1ec6
62 44f683a84163b3523afe57c2e008bc8c
67 735b90b4568125ed6c3f678819b6e058
14 aab3238922bcc25a6f606eb525ffdc56
39 d67d8ab4f4c10bf22aa353e27879133c
32 6364d3f0f495b6ab9dcf8d3b5c6e0b01
100 f899139df5e1059396431415e770c6dd
62 44f683a84163b3523afe57c2e008bc8c
60 072b030ba126b2f4b2374f342be9ed44
42 a1d0c6e83f027327d8461063f4ac58a6
68 a3f390d88e4c41f2747bfa2f1b5f87db
90 8613985ec49eb8f757ae6439e879bb2a
87 c7e1249ffc03eb9ded908c236bd1996d
29 6ea9ab1baa0efb9e19094440c317e21b
11 6512bd43d9caa6e02c990b0a82652dca
70 7cbbc409ec990f19c78c75bd1e06f215
67 735b90b4568125ed6c3f678819b6e058
79 d1fe173d08e959397adf34b1d77e88d7
30 34173cb38f07f89ddbebc2ac9128303f
73 d2ddea18f00665ce8623e36bd4e3c7c5
44 f7177163c833dff4b38fc8d2872f1ec6
41 3416a75f4cea9109507cacd8e2f2aefc
84 68d30a9594728bc39aa24be94b319d21
10 d3d9446802a44259755d38e6d163e820
39 d67d8ab4f4c10bf22aa353e2

97 e2ef524fbf3d9fe611d5a8e90fefdc9c
34 e369853df766fa44e1ed0ff613f563bd
1 c4ca4238a0b923820dcc509a6f75849b
61 7f39f8317fbdb1988ef4c628eba02591
50 c0c7c76d30bd3dcaefc96f40275bdc0a
14 aab3238922bcc25a6f606eb525ffdc56
35 1c383cd30b7c298ab50293adfecb7b18
89 7647966b7343c29048673252e490f736
6 1679091c5a880faf6fb5e6087eb1b2dc
47 67c6a1e7ce56d3d6fa748ab6d9af3fd7
21 3c59dc048e8850243be8079a5c74d079
45 6c8349cc7260ae62e3b1396831a8398f
65 fc490ca45c00b1249bbe3554a4fdf6fb
22 b6d767d2f8ed5d21a44b0e5886680cb9
49 f457c545a9ded88f18ecee47145a72c0
84 68d30a9594728bc39aa24be94b319d21
35 1c383cd30b7c298ab50293adfecb7b18
91 54229abfcfa5649e7003b83dd4755294
23 37693cfc748049e45d87b8c7d8b9aacd
6 1679091c5a880faf6fb5e6087eb1b2dc
52 9a1158154dfa42caddbd0694a4e9bdc8
68 a3f390d88e4c41f2747bfa2f1b5f87db
36 19ca14e7ea6328a42e0eb13d585e4c22
25 8e296a067a37563370ded05f5a3bf3ec
10 d3d9446802a44259755d38e6d163e820
55 b53b3a3d6ab90ce0268229151c9bde11
6 1679091c5a880faf6fb5e6087eb1b2dc
74 ad61ab143223efbc24c7d2583be69

...

64 ea5d2f1c4608232e07d3aa3d998e5135
62 44f683a84163b3523afe57c2e008bc8c
40 d645920e395fedad7bbbed0eca3fe2e0
82 9778d5d219c5080b9a6a17bef029331c
29 6ea9ab1baa0efb9e19094440c317e21b
61 7f39f8317fbdb1988ef4c628eba02591
11 6512bd43d9caa6e02c990b0a82652dca
84 68d30a9594728bc39aa24be94b319d21
9 45c48cce2e2d7fbdea1afc51c7c6ad26
96 26657d5ff9020d2abefe558796b99584
81 43ec517d68b6edd3015b3edc9a11367b
40 d645920e395fedad7bbbed0eca3fe2e0
74 ad61ab143223efbc24c7d2583be69251
19 1f0e3dad99908345f7439f8ffabdffc4
90 8613985ec49eb8f757ae6439e879bb2a
68 a3f390d88e4c41f2747bfa2f1b5f87db
88 2a38a4a9316c49e5a833517c45d31070
24 1ff1de774005f8da13f42943881c655f
33 182be0c5cdcd5072bb1864cdee4d3d6e
84 68d30a9594728bc39aa24be94b319d21
73 d2ddea18f00665ce8623e36bd4e3c7c5
37 a5bfc9e07964f8dddeb95fc584cd965d
60 072b030ba126b2f4b2374f342be9ed44
15 9bf31c7ff062936a96d3c8bd1f8f2ff3
2 c81e728d9d4c2f636f067f89cc14862c
79 d1fe173d08e959397adf34b1d77e88d7
80 f033ab37c30201f73f142449d037028d
44 f7177163c833dff4b38fc8d2872

21 3c59dc048e8850243be8079a5c74d079
61 7f39f8317fbdb1988ef4c628eba02591
98 ed3d2c21991e3bef5e069713af9fa6ca
36 19ca14e7ea6328a42e0eb13d585e4c22
28 33e75ff09dd601bbe69f351039152189
49 f457c545a9ded88f18ecee47145a72c0
64 ea5d2f1c4608232e07d3aa3d998e5135
74 ad61ab143223efbc24c7d2583be69251
75 d09bf41544a3365a46c9077ebb5e35c3
92 92cc227532d17e56e07902b254dfad10
96 26657d5ff9020d2abefe558796b99584
63 03afdbd66e7929b125f8597834fa83a4
82 9778d5d219c5080b9a6a17bef029331c
32 6364d3f0f495b6ab9dcf8d3b5c6e0b01
3 eccbc87e4b5ce2fe28308fd9f2a7baf3
82 9778d5d219c5080b9a6a17bef029331c
12 c20ad4d76fe97759aa27a0c99bff6710
13 c51ce410c124a10e0db5e4b97fc2af39
63 03afdbd66e7929b125f8597834fa83a4
20 98f13708210194c475687be6106a3b84
54 a684eceee76fc522773286a895bc8436
100 f899139df5e1059396431415e770c6dd
14 aab3238922bcc25a6f606eb525ffdc56
29 6ea9ab1baa0efb9e19094440c317e21b
21 3c59dc048e8850243be8079a5c74d079
13 c51ce410c124a10e0db5e4b97fc2af39
1 c4ca4238a0b923820dcc509a6f75849b
86 93db85ed909c13838ff95ccfa9

90 8613985ec49eb8f757ae6439e879bb2a
31 c16a5320fa475530d9583c34fd356ef5
82 9778d5d219c5080b9a6a17bef029331c
49 f457c545a9ded88f18ecee47145a72c0
66 3295c76acbf4caaed33c36b1b5fc2cb1
20 98f13708210194c475687be6106a3b84
12 c20ad4d76fe97759aa27a0c99bff6710
5 e4da3b7fbbce2345d7772b0674a318d5
94 f4b9ec30ad9f68f89b29639786cb62ef
93 98dce83da57b0395e163467c9dae521b
80 f033ab37c30201f73f142449d037028d
91 54229abfcfa5649e7003b83dd4755294
17 70efdf2ec9b086079795c442636b55fb
86 93db85ed909c13838ff95ccfa94cebd9
87 c7e1249ffc03eb9ded908c236bd1996d
76 fbd7939d674997cdb4692d34de8633c4
35 1c383cd30b7c298ab50293adfecb7b18
55 b53b3a3d6ab90ce0268229151c9bde11
4 a87ff679a2f3e71d9181a67b7542122c
62 44f683a84163b3523afe57c2e008bc8c
25 8e296a067a37563370ded05f5a3bf3ec
54 a684eceee76fc522773286a895bc8436
20 98f13708210194c475687be6106a3b84
74 ad61ab143223efbc24c7d2583be69251
14 aab3238922bcc25a6f606eb525ffdc56
37 a5bfc9e07964f8dddeb95fc584cd965d
36 19ca14e7ea6328a42e0eb13d585e4c22
80 f033ab37c30201f73f142449d03

13 c51ce410c124a10e0db5e4b97fc2af39
100 f899139df5e1059396431415e770c6dd
37 a5bfc9e07964f8dddeb95fc584cd965d
69 14bfa6bb14875e45bba028a21ed38046
95 812b4ba287f5ee0bc9d43bbf5bbe87fb
5 e4da3b7fbbce2345d7772b0674a318d5
51 2838023a778dfaecdc212708f721b788
69 14bfa6bb14875e45bba028a21ed38046
81 43ec517d68b6edd3015b3edc9a11367b
29 6ea9ab1baa0efb9e19094440c317e21b
51 2838023a778dfaecdc212708f721b788
72 32bb90e8976aab5298d5da10fe66f21d
76 fbd7939d674997cdb4692d34de8633c4
41 3416a75f4cea9109507cacd8e2f2aefc
35 1c383cd30b7c298ab50293adfecb7b18
71 e2c420d928d4bf8ce0ff2ec19b371514
55 b53b3a3d6ab90ce0268229151c9bde11
92 92cc227532d17e56e07902b254dfad10
19 1f0e3dad99908345f7439f8ffabdffc4
35 1c383cd30b7c298ab50293adfecb7b18
100 f899139df5e1059396431415e770c6dd
62 44f683a84163b3523afe57c2e008bc8c
98 ed3d2c21991e3bef5e069713af9fa6ca
14 aab3238922bcc25a6f606eb525ffdc56
20 98f13708210194c475687be6106a3b84
51 2838023a778dfaecdc212708f721b788
11 6512bd43d9caa6e02c990b0a82652dca
32 6364d3f0f495b6ab9dcf8d3b

56 9f61408e3afb633e50cdf1b20de6f466
75 d09bf41544a3365a46c9077ebb5e35c3
100 f899139df5e1059396431415e770c6dd
60 072b030ba126b2f4b2374f342be9ed44
89 7647966b7343c29048673252e490f736
30 34173cb38f07f89ddbebc2ac9128303f
23 37693cfc748049e45d87b8c7d8b9aacd
73 d2ddea18f00665ce8623e36bd4e3c7c5
80 f033ab37c30201f73f142449d037028d
76 fbd7939d674997cdb4692d34de8633c4
47 67c6a1e7ce56d3d6fa748ab6d9af3fd7
6 1679091c5a880faf6fb5e6087eb1b2dc
71 e2c420d928d4bf8ce0ff2ec19b371514
9 45c48cce2e2d7fbdea1afc51c7c6ad26
3 eccbc87e4b5ce2fe28308fd9f2a7baf3
44 f7177163c833dff4b38fc8d2872f1ec6
72 32bb90e8976aab5298d5da10fe66f21d
79 d1fe173d08e959397adf34b1d77e88d7
9 45c48cce2e2d7fbdea1afc51c7c6ad26
100 f899139df5e1059396431415e770c6dd
5 e4da3b7fbbce2345d7772b0674a318d5
82 9778d5d219c5080b9a6a17bef029331c
59 093f65e080a295f8076b1c5722a46aa2
11 6512bd43d9caa6e02c990b0a82652dca
35 1c383cd30b7c298ab50293adfecb7b18
32 6364d3f0f495b6ab9dcf8d3b5c6e0b01
38 a5771bce93e200c36f7cd9dfd0e5deaa
51 2838023a778dfaecdc212708f721